# Warning
this data is for educational use only and does not represent all breeds or species equally

In [1]:
from collections import defaultdict

In [2]:
import csv
def yieldcsv(file):
    with open(file,'r', encoding='utf-8-sig') as f:
        DictReader_obj = csv.DictReader(f)
        for item in DictReader_obj:
            yield(dict(item))

In [49]:
def chain(*iterables):
    for it in iterables:
        for element in it:
            yield element

In [3]:
def buildKeyValueFromCsv(file,key_item,value_item):
    out ={}
    for line in yieldcsv(file):
        
        if type(key_item) == list:
            key = []
            for i in key_item:
                key.append(line[i])
            key = tuple(key)
        else:
            key = line[key_item]
            
        if type(value_item) == list:
            value = []
            for i in value_item:
                value.append(line[i])
            value = tuple(value)
        if value_item is None:
            value = line
        else:
            value = line[value_item]
            
        out[key] = value
    return out

In [50]:
breed_lookup = buildKeyValueFromCsv('Breeds.csv',['sysSpeciesID','sysBreedsID'],'Breed')
sizes_lookup = buildKeyValueFromCsv('Sizes.csv',['sysSpeciesID','sysSizeChoicesID'],'SizeType')
species_lookup = buildKeyValueFromCsv('Species.csv','sysSpeciesID','Species')
nonpits = buildKeyValueFromCsv('NonPits.csv','tblAnimalsID',None)
pits = buildKeyValueFromCsv('Pits.csv','tblAnimalsID',None)
masterlist = {}
masterlist.update(pits)
masterlist.update(nonpits)
triggers = ['bite','aggressive','fight','bitten','tranq','attacked','killed']
matchedBreeds=defaultdict(list)

In [72]:
matchedSpecies=defaultdict(int)
for line in yieldcsv('PitsNotes.csv'):
    if any([v in line['Note'] for v in triggers]):
        if line['tblAnimalsID'] not in masterlist:
            continue
        animal = masterlist[line['tblAnimalsID']]
        matchedSpecies[animal['sysSpeciesID']]+=1
    
for line in yieldcsv('NonPitNotes.csv'):
    if any([v in line['Note'] for v in triggers]):
        if line['tblAnimalsID'] not in masterlist:
            continue
        animal = masterlist[line['tblAnimalsID']]
        matchedSpecies[animal['sysSpeciesID']]+=1
    
for item in sorted(matchedSpecies,key=lambda a: matchedSpecies[a]):
    name = "unknown"
    if item in species_lookup:
        name = species_lookup[item]
    print(name, matchedSpecies[item])
    

In [66]:
f =0
nf=0
for line in yieldcsv('PitsNotes.csv'):
    if any([v in line['Note'] for v in triggers]):
        f +=1
        if line['tblAnimalsID'] not in masterlist:
            continue
        animal = masterlist[line['tblAnimalsID']]
        if animal['sysSpeciesID'] != "1":
            continue
        if animal['sysBreedSecondaryID'] =='NULL':
            secondary = ""
        else:
            if (animal['sysSpeciesID'],animal['sysBreedSecondaryID']) not in breed_lookup:
                secondary = "Unknown"
            else:
                secondary = breed_lookup[(animal['sysSpeciesID'],animal['sysBreedSecondaryID'])]
        if animal['sysBreedPrimaryID'] =="NULL":
            primary = ""
        else:
            primary = breed_lookup[(animal['sysSpeciesID'],animal['sysBreedPrimaryID'])]
        breed = primary +" "+secondary
        matchedBreeds[breed].append(line)
    else:
        nf +=1
print(f'{f} items match keywords')
print(f'{nf} items do not match  keywords')

In [67]:
(f/(nf+f))*100

In [68]:
f =0
nf=0
for line in yieldcsv('NonPitNotes.csv'):
    if any([v in line['Note'] for v in triggers]):
        f +=1
        if line['tblAnimalsID'] not in masterlist:
            continue
        animal = masterlist[line['tblAnimalsID']]
        if animal['sysSpeciesID'] != "1":
            continue
        if animal['sysBreedSecondaryID'] =='NULL':
            secondary = ""
        else:
            secondary = breed_lookup[(animal['sysSpeciesID'],animal['sysBreedSecondaryID'])]
        if animal['sysBreedPrimaryID'] =="NULL":
            primary = ""
        else:
            primary = breed_lookup[(animal['sysSpeciesID'],animal['sysBreedPrimaryID'])]
        breed = primary +" "+secondary
        matchedBreeds[breed].append(line)
    else:
        nf +=1
print(f'{f} items match keywords')
print(f'{nf} items do not match keywords')

In [69]:
(f/(nf+f))*100

In [70]:
for i in sorted(matchedBreeds,key=lambda a: len(matchedBreeds[a])):
    print(i.ljust(len(max(matchedBreeds.keys(),key=len)),' '), '#'*len(matchedBreeds[i]))